# Traffic Sign Recognition

This is a completed working classification network for the german traffic sign recognition database.

**First We will load all the required libraries:**

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

"all set!"

# Loading the data

the data is pre-split into training and testing data. It contains the following:
- csv files for meta-data. All we care about from here is the class id, and path to the file.
- The training images. These are organized by their type into different folders
- The Test images. These are all together in the same folder.

**Let's load the data:**

In [ ]:
# Load the meta data
train_data = pd.read_csv('/kaggle/input/gtsrb-german-traffic-sign/Train.csv')
test_data = pd.read_csv('/kaggle/input/gtsrb-german-traffic-sign/Test.csv')

# Load the labels
train_labels = train_data['ClassId']
test_labels = test_data['ClassId']

# Load the images
train_image_paths = train_data['Path'].apply(lambda x: '/kaggle/input/gtsrb-german-traffic-sign/' + x).tolist()
test_image_paths = test_data['Path'].apply(lambda x: '/kaggle/input/gtsrb-german-traffic-sign/' + x).tolist()

# Test output to make sure we're all good
train_images.shape

**Now we'll load and create the actual images:**

In [ ]:
# three things are happening in these lines:
# 1. We are loading the images based of the file paths read in the previous cell.
# 2. We are changing the images from cv2's default blue green red, to red green blue. 
# This doesn't change training, but if we were to view the images the colors would be wrong.
# 3. scaling to 32 x 32 pixels. This will make training easier
train_images = np.array([cv2.cvtColor(cv2.resize(cv2.imread(fname), (32, 32), interpolation = cv2.INTER_AREA), cv2.COLOR_BGR2RGB) for fname in train_image_paths])
test_images = np.array([cv2.cvtColor(cv2.resize(cv2.imread(fname), (32, 32), interpolation = cv2.INTER_AREA), cv2.COLOR_BGR2RGB) for fname in test_image_paths])

There's a couple more things we need to do, but we're almost ready.

**Let's view an image to see how it looks:**

In [ ]:
plt.figure()
plt.imshow(train_images[101], cmap='gray')
plt.colorbar()
plt.grid(False)
plt.show()

It should be looking pretty good now, but the data isn't quite ready! This this model we will be using a grayscale image, so we need to convert to that. We are also going to want to scale the values down. Right now they are 0-255, but if we scale it to 0-1 it will train quite a bit faster.

**Let's do both of those things now:**

In [ ]:
train_images = np.sum(train_images/3, axis=3, keepdims=True)
train_images = train_images / 255.0

test_images = np.sum(test_images/3, axis=3, keepdims=True)
test_images = test_images / 255.0

## Training

Now that the data is all ready, we can start training!

For this I am creating a convalutional nerual network with two convalutional layers, followed by two dense layers. Feel free to try different values for everything to see if you can get better results!

**First we will need to create our model, then we will start training:**

In [ ]:
# Create the model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters = 6, kernel_size = (5, 5), strides=(1, 1), padding='valid', 
                        activation='relu', data_format = 'channels_last', input_shape = (32, 32, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(16, (5, 5), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(43, activation='softmax')
])

model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Train
conv = model.fit(train_images, train_labels, batch_size=128, epochs=10, validation_data=(test_images, test_labels))

In [ ]:
# Graph Accuracy and Loss to see how we did:

acc = [conv.history['accuracy'], conv.history['val_accuracy']]
loss = [conv.history['loss'], conv.history['val_loss']]

epoch = range(10)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epoch, acc[0], label='Training Accuracy')
plt.plot(epoch, acc[1], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epoch, loss[0], label='Training Loss')
plt.plot(epoch, loss[1], label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()